In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats


#Read in the data
features = pd.read_csv('CLINICAL Omental_mets_tumor_nuc_ft_WSI_aggregated-matched with NanoString.csv')
features = features.iloc[:, [3]].join(features.iloc[:, 41:])
gene_expression = pd.read_excel('NanoString_Log2_transformed_data.xlsx')

In [4]:

'''#Filter by highest 2/3 gene expression
gene_expression['median'] = gene_expression.iloc[:, 1:].median(axis=1, skipna=True)

cutoff = gene_expression['median'].quantile(1/3)

filtered_gene_expression = gene_expression[gene_expression['median'] > cutoff]

filtered_gene_expression = filtered_gene_expression.drop(columns=['median'])

sorted_columns = [filtered_gene_expression.columns[0]] + sorted(filtered_gene_expression.columns[1:])
filtered_gene_expression = filtered_gene_expression[sorted_columns]
filtered_gene_expression.reset_index(drop=True, inplace=True)
gene_expression = filtered_gene_expression'''


#Reorder to make sure that we are correlating correctly
Gene_expression_IDs_list = gene_expression.columns.tolist()
Gene_expression_IDs_list= Gene_expression_IDs_list[1:]
features = features[features.iloc[:, 0].isin(Gene_expression_IDs_list)]

features

,Patient_Deidentified_ID,minimum_N_F1,1thPercentile_N_F1,5thPercentile_N_F1,10thPercentile_N_F1,25thPercentile_N_F1,median_N_F1,mean_N_F1,75thPercentile_N_F1,90thPercentile_N_F1,...,10thPercentile_N_F63,25thPercentile_N_F63,median_N_F63,mean_N_F63,75thPercentile_N_F63,90thPercentile_N_F63,95thPercentile_N_F63,99thPercentile_N_F63,maximum_N_F63,entropy_N_F63
0,896,0.400978,0.853503,0.933798,0.952206,0.964706,0.971993,0.966456,0.977143,0.980836,...,43.512798,51.838332,63.441635,75.943627,80.091173,105.362655,132.901175,283.947709,10126.022362,6.110834
1,12019,0.410828,0.832373,0.916667,0.942568,0.963121,0.971963,0.963986,0.977149,0.980695,...,47.837224,54.567590,64.191911,72.039025,77.397674,97.085433,118.586567,221.889901,4135.549201,6.239679
2,12020,0.263613,0.661603,0.830582,0.900169,0.951684,0.969264,0.948870,0.976543,0.980535,...,54.706055,62.658142,73.409021,78.685299,87.222334,104.690063,119.745245,172.403582,5990.252493,6.767663
3,12051,0.391304,0.800000,0.908673,0.938849,0.961059,0.971173,0.961834,0.977077,0.980797,...,50.194045,58.501054,71.492600,88.901895,93.083623,132.522583,183.204861,376.525986,3583.272302,6.377369
4,12073,0.263501,0.846260,0.946942,0.962145,0.971214,0.976487,0.971063,0.980146,0.982794,...,51.780683,58.297554,67.043288,71.726298,78.447043,92.911192,105.648138,158.623439,1941.372848,6.765748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,44496,0.364221,0.829268,0.919776,0.945355,0.964187,0.972441,0.964739,0.977507,0.980940,...,42.671904,48.475871,56.190457,58.733785,65.717251,76.704938,84.894026,108.271711,10069.062900,7.061566
148,44513,0.406667,0.811763,0.908508,0.938889,0.961933,0.971429,0.962395,0.976932,0.980626,...,44.794718,51.091797,60.349019,69.083099,73.660090,92.630496,111.531262,205.662407,7164.436060,6.562858
149,44535,0.453395,0.796873,0.903673,0.936672,0.961313,0.971119,0.961385,0.976866,0.980714,...,54.985580,63.650972,75.838801,82.818504,91.426084,110.315258,125.820575,179.523825,20853.832267,6.236193
150,45180,0.489796,0.863477,0.928228,0.946497,0.960452,0.968815,0.963586,0.974777,0.979215,...,38.561452,43.603372,50.994636,56.853652,61.889664,77.276442,92.392689,156.741611,9161.185248,6.151512


In [5]:
# Reorder rows based on gene_expression_IDs_list
features = features.set_index(features.columns[0])  # Set first column as the index
features = features.reindex(Gene_expression_IDs_list)  # Reorder based on gene_expression_IDs_list
features = features.reset_index()  # Reset index if needed

features


,Patient_Deidentified_ID,minimum_N_F1,1thPercentile_N_F1,5thPercentile_N_F1,10thPercentile_N_F1,25thPercentile_N_F1,median_N_F1,mean_N_F1,75thPercentile_N_F1,90thPercentile_N_F1,...,10thPercentile_N_F63,25thPercentile_N_F63,median_N_F63,mean_N_F63,75thPercentile_N_F63,90thPercentile_N_F63,95thPercentile_N_F63,99thPercentile_N_F63,maximum_N_F63,entropy_N_F63
0,14171,0.536657,0.803053,0.910559,0.945679,0.965863,0.973088,0.964494,0.977984,0.981481,...,47.375203,56.970839,73.893570,93.945934,104.983324,152.953835,201.822340,373.884943,6354.132067,5.469852
1,14318,0.450685,0.821055,0.909698,0.938531,0.959459,0.969466,0.961147,0.975610,0.979798,...,44.892061,52.133798,63.084948,71.550900,78.382500,99.212354,118.209517,212.184160,12984.368039,6.237133
2,14233,0.308623,0.864328,0.939692,0.955720,0.966960,0.973561,0.968474,0.978114,0.981405,...,47.094997,53.049510,61.181345,65.275688,71.565479,84.262884,94.962919,135.630303,11911.174064,6.939091
3,14282,0.309666,0.871741,0.943452,0.957921,0.968439,0.974592,0.969771,0.978858,0.981928,...,48.339420,54.827150,63.342510,66.348145,73.754223,85.928502,95.630556,126.833080,2688.226610,7.053317
4,14566,0.417077,0.871186,0.942857,0.958258,0.968519,0.974480,0.969794,0.978805,0.982018,...,38.694145,43.552443,50.363536,53.316111,59.264028,70.159314,78.869443,104.598880,4465.940462,7.315344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,15661,0.374346,0.814591,0.914691,0.944444,0.964912,0.973077,0.964610,0.978032,0.981481,...,40.551364,45.100066,51.364072,55.817530,59.664515,70.710371,81.648094,133.007378,3545.605043,6.903129
145,896,0.400978,0.853503,0.933798,0.952206,0.964706,0.971993,0.966456,0.977143,0.980836,...,43.512798,51.838332,63.441635,75.943627,80.091173,105.362655,132.901175,283.947709,10126.022362,6.110834
146,21179,0.567901,0.879723,0.938617,0.951852,0.962591,0.969925,0.965867,0.975482,0.979730,...,36.770103,41.280243,47.859306,55.107880,57.089823,69.941133,82.710272,176.116588,2658.706066,6.110183
147,19395,0.482788,0.855110,0.929961,0.950226,0.964187,0.971681,0.965885,0.976852,0.980624,...,37.137596,41.247224,46.732193,49.674312,53.573652,61.956749,69.148964,98.177294,16831.626106,6.625122


In [6]:
gene_expression

,Patient ID,14171,14318,14233,14282,14566,14676,13553,12638,14785,...,24477,24459,24450,23099,44535,15661,896,21179,19395,22859
0,A2M,9.609920,9.045041,9.304047,11.098516,10.521699,8.889581,9.666569,8.884051,11.730628,...,9.470973,8.777730,9.027375,8.325286,10.993348,9.789774,9.526210,10.151786,8.875062,11.230119
1,AAMP,10.296721,9.826199,10.697558,9.640118,10.324383,9.180953,10.616586,10.154324,10.546528,...,10.187835,9.963373,10.556343,9.834487,9.576541,10.721031,9.636987,9.792221,10.112231,10.210915
2,ABI3BP,8.482284,7.649687,6.767920,5.875780,6.904484,6.093391,7.148426,7.487760,6.142413,...,6.219362,4.444270,8.042371,9.276450,8.663949,7.572056,7.095503,8.404205,5.946497,7.932451
3,ABL1,8.643966,8.779312,7.936572,8.772197,8.153319,8.924185,8.623873,8.186754,7.807211,...,8.553453,8.312433,9.575200,7.979330,8.272180,8.334676,8.757823,8.976895,9.285369,8.065452
4,ACKR1,6.938681,4.807355,4.807355,6.466732,5.741230,4.807355,6.617675,5.806489,5.388851,...,5.080930,4.807355,4.807355,4.807355,7.772160,5.762519,7.890126,6.020982,4.807355,5.863661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,ZEB2,9.159569,9.110927,9.987193,8.784602,8.983279,8.978853,7.725332,8.646595,8.160426,...,8.922168,7.780835,8.869409,7.573571,9.526186,8.797143,8.867773,9.680324,8.070014,9.112361
1061,ZFPM2,6.555049,5.983450,5.225352,5.203593,5.081510,5.508429,6.370862,4.536675,4.568032,...,5.412104,4.321928,5.329841,5.956521,7.513017,4.321928,5.991862,6.211596,4.724105,4.321928
1062,ZFYVE16,8.506922,8.894484,8.896544,8.192096,8.170977,8.470456,8.048923,7.965149,7.830167,...,8.136120,7.497612,8.295861,8.589389,8.642268,7.919995,8.736503,8.437627,7.907432,8.196135
1063,ZFYVE9,8.537180,8.305378,7.483171,7.719115,8.503309,8.093497,8.296824,8.703904,7.904966,...,7.823940,8.053329,8.656925,7.926948,7.880869,8.086667,8.014746,7.989139,8.363259,8.569096


In [ ]:
results = []
# Get the number of columns in the features DataFrame
feature_count = features.shape[1]

# Iterate over each column in the features DataFrame
for i, col_name in enumerate(features.columns[:1], start=1):
    print(col_name)
    if "Patient_Deidentified_ID" in col_name:
        continue  # Skip to the next iteration
    
    col_data = features[col_name].values
    
    # Iterate over each row in the gene expression DataFrame
    for index, row in gene_expression.iterrows():
        gene_name = row.iloc[0]
        expression_values = row.iloc[1:].values
        expression_values = pd.to_numeric(expression_values, errors='coerce')
        print("Expression values", expression_values[:10])
        print("Col data", col_data[:10])

        # Check if the data is constant
        if len(np.unique(col_data)) == 1 or len(np.unique(expression_values)) == 1:
            continue

        # Calculate Spearman correlation and p-value
        spearman_corr_value, spearman_p_value = stats.spearmanr(expression_values, col_data, nan_policy='omit')

        # Append the results to the list
        results.append([gene_name, col_name, spearman_corr_value, spearman_p_value])

    # Print progress every 10 features
    if i % 2 == 0 or i == feature_count:
        print(f"Number of features done: {i}")


Patient_Deidentified_ID
minimum_N_F1
Expression values [ 9.6099204   9.045041    9.30404735 11.09851641 10.52169936  8.88958062
  9.66656914  8.88405082 11.73062773  8.68964581]
Col data [0.53665689 0.45068493 0.3086233  0.30966638 0.41707718 0.37611056
 0.40026954 0.47562426 0.39838337 0.40974026]
Expression values [10.29672123  9.82619889 10.69755839  9.64011838 10.32438309  9.18095342
 10.6165856  10.15432447 10.54652795  9.91567028]
Col data [0.53665689 0.45068493 0.3086233  0.30966638 0.41707718 0.37611056
 0.40026954 0.47562426 0.39838337 0.40974026]
Expression values [8.48228361 7.6496873  6.76791959 5.87578006 6.9044841  6.09339115
 7.14842574 7.48775966 6.14241344 5.94906797]
Col data [0.53665689 0.45068493 0.3086233  0.30966638 0.41707718 0.37611056
 0.40026954 0.47562426 0.39838337 0.40974026]
Expression values [8.64396583 8.77931226 7.93657199 8.77219711 8.15331895 8.92418542
 8.62387271 8.18675408 7.80721065 8.11005416]
Col data [0.53665689 0.45068493 0.3086233  0.30966638

In [25]:

# Convert the collected results to a DataFrame
final_result_df = pd.DataFrame(results, columns=['Gene Name', 'Feature Name', 'Spearman Correlation', 'Spearman P-value'])

# Save the DataFrame to a single CSV file
# final_result_df.to_csv('gene_expression_nanostring_WSI.csv', index=False)

final_result_df


,Gene Name,Feature Name,Spearman Correlation,Spearman P-value
0,A2M,minimum_N_F1,0.086621,0.293529
1,AAMP,minimum_N_F1,0.025652,0.756152
2,ABI3BP,minimum_N_F1,0.144633,0.078429
3,ABL1,minimum_N_F1,0.076336,0.354809
4,ACKR1,minimum_N_F1,0.155701,0.057940
...,...,...,...,...
9580,ZEB2,90thPercentile_N_F1,-0.120168,0.144349
9581,ZFPM2,90thPercentile_N_F1,-0.244138,0.002695
9582,ZFYVE16,90thPercentile_N_F1,-0.335372,0.000029
9583,ZFYVE9,90thPercentile_N_F1,0.014025,0.865194
